In [20]:
import pandas as pd

pos = pd.read_csv('positif.csv')
neg = pd.read_excel('negatif.xlsx')

In [21]:
pos['label'] = 'E'
neg['label'] = '.'
pos['actual'] = ['E'*len(pos['Epitope - Name'][i]) for i in range(len(pos))]
neg['actual'] = ['.'*len(neg['Epitope - Name'][i]) for i in range(len(neg))]

In [22]:
sel_col = ['Epitope - Starting Position','Epitope - Ending Position','Epitope - Name','label','actual']

In [23]:
pos = pos[sel_col]
neg = neg[sel_col]
df1_merge = pd.concat([pos,neg])
df1_merge = df1_merge.sample(frac=1).reset_index(drop=True)

In [24]:
def protein_preprocessing(sequence):
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"  # Daftar karakter asam amino yang diperbolehkan

    processed_sequence = ""
    for char in sequence:
        if char.upper() in amino_acids:
            processed_sequence += char

    return processed_sequence

In [25]:
df1_merge['Epitope - Name'] = [protein_preprocessing(df1_merge['Epitope - Name'][i]) for i in range(len(df1_merge))]

In [26]:
df1_merge.isnull().sum()

Epitope - Starting Position    12
Epitope - Ending Position      12
Epitope - Name                  0
label                           0
actual                          0
dtype: int64

In [6]:
df1_merge

,Epitope - Starting Position,Epitope - Ending Position,Epitope - Name,label,actual
0,4951.0,4965.0,GVSICSTMTNRQFHQ,.,...............
1,4891.0,4906.0,DKSAGFPFNKWGKARL,.,................
2,201.0,215.0,SSRGTSPARMAGNGG,.,...............
3,2805.0,2820.0,EIIGYKAIDGGVTRDI,E,EEEEEEEEEEEEEEEE
4,3661.0,3675.0,MRIMTWLDMVDTSLS,.,...............
...,...,...,...,...,...
22249,1053.0,1068.0,KPTVVVNAANVYLKHG,.,................
22250,2451.0,2465.0,DTFCAGSTFISDEVA,.,...............
22251,1660.0,1674.0,QVNGLTSIKWADNNC,E,EEEEEEEEEEEEEEE
22252,5831.0,5846.0,RKAVFISPYNSQNAVA,.,................


In [79]:
def amino_epitop(amino,start,end,con):
    amino = list(amino)
    if con == 'E':
        epitop = ['E'] * len(amino)
    elif con == '.':
        epitop = ['.'] * len(amino)
    step = [start+i for i in range(0,len(amino))]
    return amino, epitop, step

In [80]:
i = 5
tes_ami, tes_epi, tes_step = amino_epitop(df1_merge['Epitope - Name'][i],df1_merge['Epitope - Starting Position'][i],df1_merge['Epitope - Ending Position'][i],df1_merge['label'][i])
print(f"amino ; {tes_ami} \n epitop : {tes_epi} \n step : {tes_step}")
print(f"amino ; {len(tes_ami)} \n epitop : {len(tes_epi)} \n step : {len(tes_step)}")

amino ; ['L', 'A', 'T', 'N', 'N', 'L', 'V', 'V', 'M', 'A', 'Y', 'I', 'T', 'G', 'G'] 
 epitop : ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E'] 
 step : [590.0, 591.0, 592.0, 593.0, 594.0, 595.0, 596.0, 597.0, 598.0, 599.0, 600.0, 601.0, 602.0, 603.0, 604.0]
amino ; 15 
 epitop : 15 
 step : 15


In [9]:
import numpy as np

def flatten(arr):
    res = []
    for x in range(len(arr)):
        for y in range(len(arr[x])):
            res.append(arr[x][y])
    return res

In [10]:
df2_merge = pd.DataFrame()

In [11]:
def hoop_woods(amino_acid):
    scale_dict = {'A': 1.0, 'R': -3.0, 'N': -3.0, 'D': -3.0, 'C': 2.0, 'Q': -3.0, 'E': -3.0, 'G': 0.0, 'H': -3.0, 'I': 1.0, 'L': 1.0, 'K': -3.0, 'M': 1.0, 'F': 1.0, 'P': -1.0, 'S': -2.0, 'T': -2.0, 'W': 1.0, 'Y': 0.0, 'V': 1.0}
    return scale_dict.get(amino_acid, 0.0)

def emini(amino_acid):
    scale_dict = {'A': 0.92, 'R': 1.02, 'N': 0.92, 'D': 0.92, 'C': 1.19, 'Q': 0.92, 'E': 0.92, 'G': 0.74, 'H': 0.92, 'I': 1.02, 'L': 1.02, 'K': 0.92, 'M': 1.02, 'F': 1.02, 'P': 0.92, 'S': 0.92, 'T': 0.92, 'W': 1.02, 'Y': 1.02, 'V': 1.02}
    return scale_dict.get(amino_acid, 0.0)

def parker(amino_acid):
    scale_dict = {'A': 1.53, 'R': -1.36, 'N': -0.12, 'D': -0.92, 'C': 1.47, 'Q': -0.71, 'E': -0.63, 'G': 0.74, 'H': -0.4, 'I': 1.61, 'L': 1.3, 'K': -1.62, 'M': 1.18, 'F': 1.38, 'P': 0.32, 'S': -0.25, 'T': -0.18, 'W': 1.37, 'Y': 0.96, 'V': 1.24}
    return scale_dict.get(amino_acid, 0.0)

def levitt(amino_acid):
    scale_dict = {'A': 1.23, 'R': 0.79, 'N': 0.68, 'D': 0.89, 'C': 0.71, 'Q': 0.97, 'E': 0.97, 'G': 1.15, 'H': 0.97, 'I': 1.18, 'L': 1.21, 'K': 0.73, 'M': 1.45, 'F': 1.13, 'P': 1.22, 'S': 0.79, 'T': 0.82, 'W': 1.08, 'Y': 1.09, 'V': 1.30}
    return scale_dict.get(amino_acid, 0.0)

In [12]:
df1_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22254 entries, 0 to 22253
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Epitope - Starting Position  22242 non-null  float64
 1   Epitope - Ending Position    22242 non-null  float64
 2   Epitope - Name               22254 non-null  object 
 3   label                        22254 non-null  object 
 4   actual                       22254 non-null  object 
dtypes: float64(2), object(3)
memory usage: 869.4+ KB


In [27]:
df1_merge = df1_merge.dropna(subset='Epitope - Starting Position')
df1_merge = df1_merge.dropna(subset='Epitope - Ending Position')
df1_merge = df1_merge.sample(frac=1).reset_index(drop=True)

In [28]:
df1_merge.isnull().sum()

Epitope - Starting Position    0
Epitope - Ending Position      0
Epitope - Name                 0
label                          0
actual                         0
dtype: int64

In [29]:
df1_merge.to_csv('dataset_type_1.csv',index=False)

In [98]:
from itertools import chain

df1 = pd.read_csv('dataset_type_1.csv')

AllAmino = []
AllEpitop = []
AllPos = []
df2_merge = pd.DataFrame()
for i in range(len(df1)):
    ami, epi, pos = amino_epitop(df1_merge['Epitope - Name'][i],df1_merge['Epitope - Starting Position'][i],df1_merge['Epitope - Ending Position'][i],df1_merge['label'][i])
    AllAmino.append(ami)
    AllEpitop.append(epi) 
    AllPos.append(pos)

print(len(AllAmino),len(AllEpitop),len(AllPos))


22242 22242 22242


In [60]:
total = 0
for x in range(len(AllPos)):
    total += len(AllPos[x])
    for y in range(len(AllPos[x])):
        if not isinstance(AllPos[x][y], int):
            print(type(AllPos[x][y]))

In [61]:
total

345305

In [52]:
len(AllPos)

22242

In [99]:
Amino = list(chain.from_iterable(AllAmino))
Epitop = list(chain.from_iterable(AllEpitop))
Pos = list(chain.from_iterable(AllPos))
#AllAmino = flatten(AllAmino)
#AllEpitop = flatten(AllEpitop)
#AllPos = flatten(AllPos)
print(len(Amino),len(Epitop),len(Pos))

345328 345328 345328


In [88]:
1 != 1 != 1

False

In [100]:
df2_merge['amino'] = Amino
df2_merge['label'] = Epitop
df2_merge['Position'] = Pos

In [96]:
df2_merge

,label,amino,position,hoop_woods,emini,parker,levitt,Position
0,S,E,1175.0,-3.0,0.92,-0.63,0.97,1175.0
1,V,E,1176.0,-3.0,0.92,-0.63,0.97,1176.0
2,V,E,1177.0,-3.0,0.92,-0.63,0.97,1177.0
3,N,E,1178.0,-3.0,0.92,-0.63,0.97,1178.0
4,I,E,1179.0,-3.0,0.92,-0.63,0.97,1179.0
...,...,...,...,...,...,...,...,...
345323,L,.,878.0,0.0,0.00,0.00,0.00,878.0
345324,A,.,879.0,0.0,0.00,0.00,0.00,879.0
345325,G,.,880.0,0.0,0.00,0.00,0.00,880.0
345326,T,.,881.0,0.0,0.00,0.00,0.00,881.0


In [101]:
df2_merge['hoop_woods'] = [hoop_woods(df2_merge['amino'][i]) for i in range(len(df2_merge))]
df2_merge['emini'] = [emini(df2_merge['amino'][i]) for i in range(len(df2_merge))]
df2_merge['parker'] = [parker(df2_merge['amino'][i]) for i in range(len(df2_merge))]
df2_merge['levitt'] = [levitt(df2_merge['amino'][i]) for i in range(len(df2_merge))]

In [102]:
df2_merge

,amino,label,Position,hoop_woods,emini,parker,levitt
0,S,E,1175.0,-2.0,0.92,-0.25,0.79
1,V,E,1176.0,1.0,1.02,1.24,1.30
2,V,E,1177.0,1.0,1.02,1.24,1.30
3,N,E,1178.0,-3.0,0.92,-0.12,0.68
4,I,E,1179.0,1.0,1.02,1.61,1.18
...,...,...,...,...,...,...,...
345323,L,.,878.0,1.0,1.02,1.30,1.21
345324,A,.,879.0,1.0,0.92,1.53,1.23
345325,G,.,880.0,0.0,0.74,0.74,1.15
345326,T,.,881.0,-2.0,0.92,-0.18,0.82


In [103]:
df2_merge.to_csv('dataset_type_2.csv',index=False)